<a href="https://colab.research.google.com/github/Dong2Yo/DATA3960_1252/blob/main/Lectures/Geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Geocoding Strategy

Addresses are text-based attributes that limit spatial analysis.
To enable mapping, distance calculations, and geographic aggregation,
addresses must be transformed into latitude and longitude coordinates.

This notebook applies the following strategy:

1. Use a public geocoding service (OpenStreetMap / Nominatim)
2. Apply rate limiting and error handling
3. Preserve original address fields
4. Flag unsuccessful geocoding attempts

Geocoding is treated as a **data enrichment step**, not a cleaning step.


### Step 1: Data Standardization (Before Geocoding)

This is mandatory before you geocode.

Typical actions:

Trim whitespace

Standardize casing (Title Case or Upper Case)

Split address into components (if possible):

Street number

Street name

Unit / suite

City

Province

Postal code

Ensure consistent city name (Edmonton, not Edm, EDMONTON, etc.)

Validate postal codes (Canadian format: A1A 1A1)

👉 Output: a clean, single-line address field like:

In [ ]:
12345 97 St NW, Edmonton, AB, T5G 1M1


###  Step 2: Practical Geocoding Options (Edmonton / Canada)

>Option A: OpenStreetMap (Recommended for Students)

Free

Good coverage for Canadian addresses

No credit card required (with reasonable limits)

Common services:

Nominatim

Photon

Pelias

⚠️ Needs:

Rate limiting

Error handling

Missing match logic

>Option B: Google Maps API

Very accurate

Requires API key + billing

Usually not ideal for class assignments

>Option C: City of Edmonton Open Data (Best Practice)

Edmonton provides:

Address point datasets

Parcel datasets

Neighborhood boundaries

👉 Pro move:

Join addresses to city-provided address points

Avoid API limits entirely

Much more defensible academically

This is gold for a capstone if available.

In [3]:
pip install geopy

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# 1. Load the data
csv_url = "https://raw.githubusercontent.com/Dong2Yo/Dataset/refs/heads/main/calgary_listings_cleaned.csv"
try:
    df = pd.read_csv(csv_url)
    print(f"Successfully loaded data from {csv_url}. Shape: {df.shape}")
except Exception as e:
    print(f"Error loading CSV: {e}")
    df = pd.DataFrame() # Create an empty DataFrame to avoid errors later

if not df.empty:
    # Rename 'Address' column to 'address' to match expected column name
    if 'Address' in df.columns:
        df.rename(columns={'Address': 'address'}, inplace=True)
        print("Renamed 'Address' column to 'address'.")

    if 'address' not in df.columns:
        print("Error: 'address' column still not found after renaming. Please check column names in your CSV.")
    else:
        # 2. Initialize Nominatim geocoder with a user_agent
        # It's good practice to provide a unique user_agent string
        geolocator = Nominatim(user_agent="my-geocoding-app")

        # 3. Apply rate limiting to avoid blocking by the service
        # Nominatim recommends 1 request per second
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=5, swallow_exceptions=True)

        # 4. Geocode the addresses
        print("Starting geocoding process. This may take a while...")
        # Apply the geocoding function
        df['location'] = df['address'].apply(geocode)

        # 5. Extract latitude, longitude, and flag unsuccessful attempts
        df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
        df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)
        df['geocoding_successful'] = df['location'].apply(lambda loc: loc is not None)

        # Clean up the 'location' column if you don't need the full object
        df = df.drop(columns=['location'])

        print("Geocoding complete.")
        print("\nDataFrame with new latitude and longitude columns:")
        print(df[['address', 'latitude', 'longitude', 'geocoding_successful']].head())
        print(f"\nNumber of successfully geocoded addresses: {df['geocoding_successful'].sum()} out of {len(df)}")
else:
    print("Cannot proceed with geocoding as the DataFrame is empty or could not be loaded.")


# Task
Geocode addresses from the CSV file "https://raw.githubusercontent.com/Dong2Yo/Dataset/refs/heads/main/calgary_listings_cleaned.csv" by creating a Python function to perform the geocoding, applying it to the DataFrame, and then displaying the head of the resulting DataFrame with latitude, longitude, and a geocoding success flag, along with the count of successfully geocoded addresses.

## Define Geocoding Transformer Function

### Subtask:
Create a new code cell to define a Python function, `geocode_dataframe`, that takes a pandas DataFrame and the name of the address column as input. This function will encapsulate the `Nominatim` geolocator initialization, `RateLimiter` application, and the logic to geocode addresses, extract latitude and longitude, and add a geocoding success flag. It will return the modified DataFrame.


**Reasoning**:
The subtask requires defining a Python function `geocode_dataframe` that encapsulates the geocoding logic, including `Nominatim` initialization, `RateLimiter` application, and extraction of geographical coordinates, as per the instructions. This code block will create that function.



In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

def geocode_dataframe(df, address_column):
    """
    Geocodes addresses in a DataFrame using Nominatim with rate limiting.

    Args:
        df (pd.DataFrame): The input DataFrame.
        address_column (str): The name of the column containing addresses.

    Returns:
        pd.DataFrame: The DataFrame with 'latitude', 'longitude', and 'geocoding_successful' columns added.
    """
    # Initialize Nominatim geocoder with a user_agent
    geolocator = Nominatim(user_agent="my-geocoding-app")

    # Apply rate limiting to avoid blocking by the service
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=5, swallow_exceptions=True)

    # Geocode the addresses and store the full location object temporarily
    df['location'] = df[address_column].apply(geocode)

    # Extract latitude, longitude, and flag unsuccessful attempts
    df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
    df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)
    df['geocoding_successful'] = df['location'].apply(lambda loc: loc is not None)

    # Drop the temporary 'location' column
    df = df.drop(columns=['location'])

    print("Geocoding function 'geocode_dataframe' defined.")
    return df

**Reasoning**:
Now that the `geocode_dataframe` function is defined, the next step is to apply it to the `df` DataFrame using the 'address' column, then display the head of the updated DataFrame and the count of successfully geocoded addresses as per the task requirements.



In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

def geocode_dataframe(df, address_column):
    """
    Geocodes addresses in a DataFrame using Nominatim with rate limiting.

    Args:
        df (pd.DataFrame): The input DataFrame.
        address_column (str): The name of the column containing addresses.

    Returns:
        pd.DataFrame: The DataFrame with 'latitude', 'longitude', and 'geocoding_successful' columns added.
    """
    # Initialize Nominatim geocoder with a user_agent
    geolocator = Nominatim(user_agent="my-geocoding-app")

    # Apply rate limiting to avoid blocking by the service
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=5, swallow_exceptions=True)

    # Geocode the addresses and store the full location object temporarily
    df['location'] = df[address_column].apply(geocode)

    # Extract latitude, longitude, and flag unsuccessful attempts
    df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
    df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)
    df['geocoding_successful'] = df['location'].apply(lambda loc: loc is not None)

    # Drop the temporary 'location' column
    df = df.drop(columns=['location'])

    print("Geocoding function 'geocode_dataframe' defined.")
    return df

# Assuming df is already loaded and 'address' is the correct column name
# (based on previous code and kernel state)
if 'address' in df.columns:
    print("Applying geocoding function...")
    df_geocoded = geocode_dataframe(df.copy(), 'address') # Use a copy to avoid modifying original df directly

    print("\nDataFrame head after geocoding:")
    print(df_geocoded[['address', 'latitude', 'longitude', 'geocoding_successful']].head())

    successful_count = df_geocoded['geocoding_successful'].sum()
    total_addresses = len(df_geocoded)
    print(f"\nNumber of successfully geocoded addresses: {successful_count} out of {total_addresses}")
else:
    print("Error: 'address' column not found in DataFrame. Cannot proceed with geocoding.")